## Intro to the datasets

**EDIT: I deleted the GitHub dataset here since the description in the doc made me confused.**

In YouTube dataset, each line contains a user identifier followed by a group identifier (separated by a tab), implying that the user is a member of the group.

**user-num**: 94238

**group-num**:30087


## Randomly select 20000 users

As is proposed in the paper of BiNE, we should extract 15000 users and reserve all their edges to be our universal dataset. Since these 2 datasets are sparser than Wikipedia, we select 20000 users here.

After selection, the edge number:

* **YouTube**: 62098

Also, we want to rename all user identifiers by adding a letter 'u' before them, and all group/project identifiers by a letter 'i' so as to distinguish them.

Note that in YouTube dataset there is one extra line, so we use `next` to skip it.

In [76]:
import numpy as np

In [37]:
a = np.arange(1,94239)
b = np.arange(1, 120868)

In [47]:
rs1 = np.random.RandomState(111)
rs2 = np.random.RandomState(222)
#users_y = rs1.randint(low=1, high=94238, size = 20000)
#users_g = rs2.randint(low=1, high=120867, size = 20000)
rs1.shuffle(a)
rs2.shuffle(b)
users_y = a[:20000]
users_g = b[:20000]

In [48]:
with open('out.youtube-groupmemberships', 'r') as fin, open('youtube_selected.dat','w') as fout:
    next(fin)
    line_num = 0
    for line in fin:
        user, group = line.strip().split(' ')
        if int(user) in users_y:
            line_num += 1
            fout.write('u' + user + ' ' + 'i' + group + '\n')    

line_num

62098

## Split training and test set

As BiNE proposed, we should take 60% edges of each user as the training set, and the other 40% as the positive samples in the test set. For the test set, we need to sample an equal number of negative samples i.e. edges that does not exist.

### First we calculate each user's edge number


In [61]:
def user_cnt(filename):
    cnt = []
    
    i = 0
    first = True
    with open(filename, 'r') as fin:
        for line in fin:
            user, group = line.strip().split(' ')
            # for the first line
            if first:
                cur = user
                first = False
                i = 0
        
            if user != cur:
                cnt.append(i) # count finish
                i = 1 # restart and count for 1
                cur = user  
            else:
                i += 1
    
        # the last entity
        cnt.append(i)
    
    return cnt

In [62]:
youtube_cnt = user_cnt('youtube_selected.dat')

### Then for each user, split 60% for Training set

Before using pandas `read_table`, I manually add a line in the file:
```
user item
```
to serve as the heading.

In [ ]:
with open('youtube_selected.dat', 'r') as fin, open('youtube_train.dat', 'w') as fout1,
open('youtube_test', 'w') as fout2:
    for line in fin:
        user, group = line.strip().split(' ')
        

In [83]:
import pandas as pd

In [84]:
f = pd.read_table('youtube_selected.dat', sep = ' ')

In [88]:
start, end = 0, 0
for i in youtube_cnt:
    start = end
    end = start + i
    x = f.iloc[start:end, :].sample(frac=1)
    
    

,user,item
2,u5,i22
1,u5,i21
0,u5,i20
